In [1]:
import os
import numpy as np
import pandas as pd 

from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold,cross_val_score

from pymer4.models import Lmer
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, RandomForestRegressor
from merf import MERF
from sklearn.pipeline import Pipeline

import optuna

from scipy.special import expit 

import plotly.express as px


In [2]:
target_label = 'binary_compound'
# Specify the summary statistics you want: 'mean', 'std', or ['mean', 'std']
summarize = ['mean', 'std']

eeg_path = '../Data/EEG/df_all_markers.csv'
sentiment_path = '../Data/Sentiment/sentiment_results_EEG.csv'

eeg_data = pd.read_csv(eeg_path)
eeg_data.rename(columns={'participant': 'subject_id'}, inplace=True)
# List of features to summarize
features = ['wSMI_1', 'wSMI_2', 'wSMI_4', 'wSMI_8', 'p_e_1', 'p_e_2',
            'p_e_4', 'p_e_8', 'k', 'se', 'msf', 'sef90', 'sef95', 'b', 'b_n', 
            'g', 'g_n', 't', 't_n', 'd', 'd_n', 'a_n', 'a']

# Summarize data per participant by the chosen summary statistics
if summarize:
    eeg_data = eeg_data.groupby(['subject_id', 'round'])[features].agg(summarize).reset_index()
    eeg_data.columns = ['_'.join(col).strip() if col[1] else col[0] for col in eeg_data.columns.values]
    features = eeg_data.columns[2:]
print(features)

sentiment_data = pd.read_csv(sentiment_path)
sentiment_data = sentiment_data[sentiment_data.subject_id.apply(lambda x: x.isnumeric())]
sentiment_data.subject_id = sentiment_data.subject_id.astype('int64')

merged = eeg_data.merge(sentiment_data, on=['subject_id','round'])
merged = merged.drop(['error'], axis = 1)
merged.dropna(inplace=True)

results_path = '../Results/Sentiment_classification' 

Index(['wSMI_1_mean', 'wSMI_1_std', 'wSMI_2_mean', 'wSMI_2_std', 'wSMI_4_mean',
       'wSMI_4_std', 'wSMI_8_mean', 'wSMI_8_std', 'p_e_1_mean', 'p_e_1_std',
       'p_e_2_mean', 'p_e_2_std', 'p_e_4_mean', 'p_e_4_std', 'p_e_8_mean',
       'p_e_8_std', 'k_mean', 'k_std', 'se_mean', 'se_std', 'msf_mean',
       'msf_std', 'sef90_mean', 'sef90_std', 'sef95_mean', 'sef95_std',
       'b_mean', 'b_std', 'b_n_mean', 'b_n_std', 'g_mean', 'g_std', 'g_n_mean',
       'g_n_std', 't_mean', 't_std', 't_n_mean', 't_n_std', 'd_mean', 'd_std',
       'd_n_mean', 'd_n_std', 'a_n_mean', 'a_n_std', 'a_mean', 'a_std'],
      dtype='object')


# Random Forest

In [20]:
#Esto lo uso para cuando me armo un modelo con todos los sujetos, asi que tengo que mergear todos los archivos individuales en uno
#X = merged[features].values  # Convert to NumPy array
#y = merged[target_label].values  # Convert to NumPy array
#groups = merged['subject_id'].values  # Convert to NumPy array

#Si quiero hacerlo intra sujeto solo necesito el archivo de cada uno
path = '/home/nicobruno/flor/Tesis/resultados/'
X = path + 'S02_auditivo_evoked_all_marker.csv'
y = path + 'subir los vectores a git'
#deberia ver que hago con groups en este caso

def objective(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_uniform('max_features', 0.1, 1.0)

    # Create a pipeline
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            random_state=42,
            n_jobs=-1
        ))
    ])

    # Cross-validation using GroupKFold
    group_kfold = GroupKFold(n_splits=5)
    scores = cross_val_score(pipeline, X, y, groups=groups, cv=group_kfold, scoring='roc_auc')

    return np.mean(scores)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Best hyperparameters
print('Best trial:', study.best_trial.params)
study.trials_dataframe().sort_values('value', ascending=False).head()

[I 2023-11-21 19:07:37,447] A new study created in memory with name: no-name-7ab2ba37-4cb9-4737-8c65-4c332e379f82
/tmp/ipykernel_43479/1348297499.py:10: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-11-21 19:07:40,466] Trial 0 finished with value: 0.5009221579686348 and parameters: {'n_estimators': 161, 'max_depth': 17, 'min_samples_leaf': 3, 'max_features': 0.36990440022035875}. Best is trial 0 with value: 0.5009221579686348.
/tmp/ipykernel_43479/1348297499.py:10: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

[I 2023-11-21 19:07:41,590] Trial 1 finished with value: 0.4235127674258109 and parameters: {'n_estimators': 53, 'max_depth': 5, 'min_samples_leaf': 6, 'max_features': 0.7508456316779147}. B

Best trial: {'n_estimators': 14, 'max_depth': 6, 'min_samples_leaf': 1, 'max_features': 0.4869297177525387}


,number,value,datetime_start,datetime_complete,duration,params_max_depth,params_max_features,params_min_samples_leaf,params_n_estimators,state
41,41,0.584433,2023-11-21 19:09:08.177659,2023-11-21 19:09:08.678786,0 days 00:00:00.501127,6,0.486930,1,14,COMPLETE
42,42,0.578315,2023-11-21 19:09:08.680770,2023-11-21 19:09:09.108936,0 days 00:00:00.428166,6,0.510714,1,10,COMPLETE
49,49,0.527214,2023-11-21 19:09:14.001362,2023-11-21 19:09:14.609093,0 days 00:00:00.607731,9,0.573399,2,20,COMPLETE
28,28,0.517401,2023-11-21 19:08:44.604797,2023-11-21 19:08:46.224674,0 days 00:00:01.619877,20,0.492932,1,76,COMPLETE
44,44,0.513958,2023-11-21 19:09:09.538823,2023-11-21 19:09:10.298799,0 days 00:00:00.759976,7,0.497785,1,28,COMPLETE


# GLMM

In [9]:

# Generate fixed effects formula dynamically
fixed_effects_formula = ' + '.join(features)

# Fitting the GLMM model
formula = f"{target_label} ~ {fixed_effects_formula} + (1|subject_id)"
print(formula)

scores = []
perm_scores_all = []

n_splits = 5
group_kfold = GroupKFold(n_splits=n_splits)

# Assuming df_mw['participant'] contains the participant IDs
groups = merged['subject_id'].values
X = merged[features]
y = merged[target_label]

for train_index, test_index in group_kfold.split(X, y, groups):
    df_train = merged.iloc[train_index]
    # df_train[marker] = StandardScaler().fit_transform(df_train[marker].values.reshape(-1, 1))
    df_test = merged.iloc[test_index]
    # df_test[marker] = StandardScaler().fit_transform(df_test[marker].values.reshape(-1, 1))
    y_test = y[test_index]
    model = Lmer(formula, data=df_train, family="binomial")
    model.fit(verbose = False, summary = False)
    

    predicted_probabilities = model.predict(df_test, use_rfx=True, verify_predictions=False)
    predicted_labels = [1 if prob > 0.5 else 0 for prob in predicted_probabilities]

    auc = roc_auc_score(y_test, predicted_probabilities)
    scores.append(auc)
    
    
    # elif metric == 'f1':
    #     f1 = f1_score(y_test, predicted_labels)
    #     scores.append(f1)
    # elif metric == 'kappa':
    #     kappa = cohen_kappa_score(y_test, predicted_labels)
    #     scores.append(kappa)
    
print(f"Mean AUC: {np.mean(scores)}")
print(f"Std AUC: {np.std(scores)}")

binary_compound ~ wSMI_1_mean + wSMI_2_mean + wSMI_4_mean + wSMI_8_mean + p_e_1_mean + p_e_2_mean + p_e_4_mean + p_e_8_mean + k_mean + se_mean + msf_mean + sef90_mean + sef95_mean + b_mean + b_n_mean + g_mean + g_n_mean + t_mean + t_n_mean + d_mean + d_n_mean + a_n_mean + a_mean + (1|subject_id)


RRuntimeError: Error in pwrssUpdate(pp, resp, tol = tolPwrss, GQmat = GQmat, compDev = compDev,  : 
  Downdated VtV is not positive definite


# MERF

In [50]:
# Initialize lists to store AUC and permutation scores
scores = []
perm_auc_scores_all = []
optimal_cutoffs = []

n_splits = 5
group_kfold = GroupKFold(n_splits=n_splits)

# Assuming df_mw['participant'] contains the participant IDs
groups = merged['subject_id']
X = merged[features]
y = merged[target_label]
Z = np.ones((X.shape[0], 1))  # Random effects design matrix
def objective(trial):
    # Hyperparameters to be optimized
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_uniform('max_features', 0.1, 1.0)

    scores = []
    
    for train_index, test_index in group_kfold.split(X, y, groups.values):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        X_train = StandardScaler().fit_transform(X_train)
        X_test = StandardScaler().fit_transform(X_test)
        
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clusters_train, clusters_test = groups.iloc[train_index], groups.iloc[test_index]
        
        # Initialize MERF with trial-suggested parameters
        merf = MERF(fixed_effects_model=RandomForestRegressor(
                    n_estimators=n_estimators,
                    max_depth=max_depth,
                    min_samples_leaf=min_samples_leaf,
                    max_features=max_features,
                    random_state=42,
                    n_jobs=-1
        ),
        gll_early_stop_threshold=1,
        max_iterations=50
        )
        
        merf.fit(X_train, Z[train_index], clusters_train, y_train)
        
        
        y_pred = merf.predict(X_test, Z[test_index], clusters_test)

        # Convert continuous outputs to probabilities
        y_pred_proba = expit(y_pred)
        
        # Find optimal cutoff point based on F1 score
        thresholds = np.linspace(0, 1, 100)
        f1_scores = [f1_score(y_test, (y_pred_proba > t).astype(int)) for t in thresholds]
        optimal_threshold = thresholds[np.argmax(f1_scores)]
        optimal_cutoffs.append(optimal_threshold)
        
        # Thresholding to get class labels
        y_pred_class = (y_pred_proba > optimal_threshold).astype(int)

        # Compute AUC for probabilities
        auc = roc_auc_score(y_test, y_pred_proba)
        scores.append(auc)

    # Average AUC over all folds
    avg_auc = np.mean(scores)
    return avg_auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters.
# Save the best trial's parameters to a file
best_params_path = os.path.join(results_path, f'sentiment_{target_label}_merf_best_params.txt')
with open(best_params_path, 'w') as file:
    for key, value in study.best_trial.params.items():
        file.write(f'{key}: {value}\n')

# print('Best trial:', study.best_trial.params)
params_df = study.trials_dataframe().sort_values('value', ascending=False).head()
params_df.to_csv(os.path.join(results_path, f'sentiment_{target_label}_merf_opt_params.csv'))
params_df

[I 2023-11-21 21:27:06,008] A new study created in memory with name: no-name-eea3ab70-33da-4c77-9b53-4996024926a5
/tmp/ipykernel_43479/3418640195.py:19: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

INFO     [merf.py:307] Training GLL is -135.48537784393915 at iteration 1.
INFO     [merf.py:307] Training GLL is -165.14497585070518 at iteration 2.
INFO     [merf.py:321] Gll -165.14497585070518 less than threshold 0.21891364572883934, stopping early ...
INFO     [merf.py:307] Training GLL is -168.06540904470972 at iteration 1.
INFO     [merf.py:307] Training GLL is -205.3829282096054 at iteration 2.
INFO     [merf.py:321] Gll -205.3829282096054 less than threshold 0.22204164067436544, stopping early ...
INFO     [merf.py:307] Training GLL is -155.83405660252274 at iteration 1.
INFO     [merf.py:307] Training GLL is -189.0046232626487 at iteration 

,number,value,datetime_start,datetime_complete,duration,params_max_depth,params_max_features,params_min_samples_leaf,params_n_estimators,state
82,82,0.713754,2023-11-21 21:32:37.915050,2023-11-21 21:32:40.825793,0 days 00:00:02.910743,14,0.196043,3,33,COMPLETE
60,60,0.708449,2023-11-21 21:31:32.604270,2023-11-21 21:31:35.399629,0 days 00:00:02.795359,10,0.166610,2,30,COMPLETE
43,43,0.699641,2023-11-21 21:30:29.721497,2023-11-21 21:30:33.610267,0 days 00:00:03.888770,2,0.218361,4,67,COMPLETE
16,16,0.697767,2023-11-21 21:28:28.692990,2023-11-21 21:28:31.648571,0 days 00:00:02.955581,3,0.227631,3,35,COMPLETE
53,53,0.695117,2023-11-21 21:31:10.194724,2023-11-21 21:31:13.033603,0 days 00:00:02.838879,3,0.234067,3,32,COMPLETE


In [51]:
best_params = study.best_trial.params

# Initialize lists to store AUC and permutation scores
scores = []
perm_auc_scores_all = []
optimal_cutoffs = []

n_splits = 5
group_kfold = GroupKFold(n_splits=n_splits)

# Assuming df_mw['participant'] contains the participant IDs
groups = merged['subject_id']
X = merged[features]
y = merged[target_label]
Z = np.ones((X.shape[0], 1))  # Random effects design matrix

# Replace with your actual file path
best_params_path = os.path.join(results_path, f'sentiment_{target_label}_merf_best_params.txt')

# Read the parameters from the file and store them in a dictionary
best_params = {}
with open(best_params_path, 'r') as file:
    for line in file:
        key, value = line.split(': ')
        best_params[key] = float(value.strip())

scores = []

for train_index, test_index in group_kfold.split(X, y, groups.values):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    X_train = StandardScaler().fit_transform(X_train)
    X_test = StandardScaler().fit_transform(X_test)
    
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clusters_train, clusters_test = groups.iloc[train_index], groups.iloc[test_index]
    
    # Create and configure the model with the best parameters
    merf = MERF(
        RandomForestRegressor(
            n_estimators=int(best_params["n_estimators"]),
            max_depth=int(best_params["max_depth"]),
            min_samples_leaf=int(best_params["min_samples_leaf"]),
            max_features=best_params["max_features"],
            random_state=42,  # Assuming you want to keep the random state fixed
            n_jobs=-1
            ),
        gll_early_stop_threshold=0.5,
        max_iterations=50
        )
    
    merf.fit(X_train, Z[train_index], clusters_train, y_train)
    
    
    y_pred = merf.predict(X_test, Z[test_index], clusters_test)

    # Convert continuous outputs to probabilities
    y_pred_proba = expit(y_pred)
    
    # Find optimal cutoff point based on F1 score
    thresholds = np.linspace(0, 1, 100)
    f1_scores = [f1_score(y_test, (y_pred_proba > t).astype(int)) for t in thresholds]
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    optimal_cutoffs.append(optimal_threshold)
    
    # Thresholding to get class labels
    y_pred_class = (y_pred_proba > optimal_threshold).astype(int)

    # Compute AUC for probabilities
    auc = roc_auc_score(y_test, y_pred_proba)
    scores.append(auc)

# Average AUC over all folds
avg_auc = np.mean(scores)
print(scores)
print(avg_auc)


INFO     [merf.py:307] Training GLL is -192.50929287021145 at iteration 1.


INFO     [merf.py:307] Training GLL is -239.12583046842894 at iteration 2.
INFO     [merf.py:321] Gll -239.12583046842894 less than threshold 0.24215214186904768, stopping early ...
INFO     [merf.py:307] Training GLL is -213.81185329818143 at iteration 1.
INFO     [merf.py:307] Training GLL is -279.55687789953436 at iteration 2.
INFO     [merf.py:321] Gll -279.55687789953436 less than threshold 0.3074900833943247, stopping early ...
INFO     [merf.py:307] Training GLL is -207.06342893339266 at iteration 1.
INFO     [merf.py:307] Training GLL is -267.09135188661463 at iteration 2.
INFO     [merf.py:321] Gll -267.09135188661463 less than threshold 0.28990113446122595, stopping early ...
INFO     [merf.py:307] Training GLL is -209.54563373313036 at iteration 1.
INFO     [merf.py:307] Training GLL is -271.64519686673475 at iteration 2.
INFO     [merf.py:321] Gll -271.64519686673475 less than threshold 0.2963534101249378, stopping early ...
INFO     [merf.py:307] Training GLL is -199.54951

[1.0, 0.5277777777777778, 0.6416666666666666, 0.5297619047619048, 0.8695652173913043]
0.7137543133195307


In [52]:
# Extract feature importances
feature_importances = merf.fe_model.feature_importances_


# Combine names and importances into a DataFrame
importances_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
importances_df.to_csv(os.path.join(results_path, f'sentiment_{target_label}_merf_feat_imp.csv'))

# Create a bar chart
fig = px.bar(importances_df, x='Importance', y='Feature', orientation='h',
             title=f'Feat Imp MERF {target_label}, AUC: {avg_auc:.3f}')

# Customize the layout
fig.update_layout(yaxis={'categoryorder': 'total ascending'},
                  xaxis_title='Importance',
                  yaxis_title='Feature')

fig.update_layout(
    width=650,
    height=1000,
#     autosize = True, 
    template = 'plotly_white',
        font=dict(
        family="Times new roman",
        size=20,
        color="black"
    ),
    xaxis = dict(
            visible=True,
            tickfont = {"size": 20},
        ),
    yaxis = dict(
        tickfont = {"size": 20},
        autorange = False,    
        automargin = True,
        range = [-1,len(features)],
        dtick = 1
        ),
    showlegend=True, 

)

# Show the plot
fig.show()

fig.write_image(os.path.join(results_path,f'feat_imp_sentiment_{target_label}.pdf'))


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

aucs = []
kf = KFold(n_splits=5)
for train, test in kf.split(my_data):
    X_train, X_test, y_train, y_test = my_data[train], my_data[test], labels[train], labels[test]
    clf = RandomForestClassifier(n_estimators=1000)
    clf.fit(X_train,y_train)
    probas = clf.predict_proba(X_test)
    aucs.append(roc_auc_score(y_test,probas))

    

